In [1]:
# import 
import pandas as pd;
import numpy as np;

In [5]:
# Datasets
X1 = pd.read_csv('../datasets/complete-data/Caltech/GISTFeature.csv',header=None);
X2 = pd.read_csv('../datasets/complete-data/Caltech/LBPFeature.csv',header=None);
print(X1.shape) # (1474, 512)
print(X2.shape) # (1474,928)
X1.head()

(1474, 512)
(1474, 928)


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.149860,0.049631,0.049833,0.030009,0.166100,0.045195,0.052351,0.032841,0.072760,0.067054,...,0.045340,0.038174,0.068873,0.088719,0.060006,0.046778,0.024452,0.050038,0.096785,0.070728
1,0.168870,0.082086,0.167050,0.109790,0.113190,0.045916,0.050687,0.031878,0.207420,0.102760,...,0.118360,0.075107,0.081991,0.046442,0.163500,0.110060,0.112140,0.085244,0.079044,0.051326
2,0.031449,0.038602,0.040515,0.064938,0.024486,0.064542,0.038623,0.055522,0.035708,0.034724,...,0.040049,0.055716,0.054746,0.068624,0.043013,0.089594,0.009888,0.060689,0.055650,0.049560
3,0.062685,0.069544,0.061907,0.038404,0.052422,0.041499,0.053826,0.048148,0.034058,0.040677,...,0.085608,0.071643,0.119500,0.058825,0.082927,0.062791,0.048369,0.044633,0.069152,0.102740
4,0.062920,0.034458,0.015966,0.022567,0.127380,0.098888,0.062020,0.031535,0.049430,0.058208,...,0.029204,0.038763,0.047977,0.071451,0.072366,0.055249,0.028656,0.041247,0.110050,0.054509


In [54]:
# Normalize
from sklearn.preprocessing import Normalizer
transformer = Normalizer();
Xn1 = transformer.transform(X1).T
Xn2 = transformer.transform(X2).T

In [55]:
np.random.seed(123)
import numpy.matlib
Ht = np.matlib.rand(V[0].shape)

In [56]:
V = [Xn1,Xn2];

In [68]:
V[0].shape  # 10x 1474
# H => 10 x 1474
# w => 1474 x 10


(512, 1474)

In [ ]:
# 512 x 1474 => (512 x 10) x (10 x 1474)
#      X     =>   W      x   H

In [87]:
# random H1,H2, W1,W2 # k = 5
H=[]
W=[]
for i in range(len(V)):
    sh = V[i].shape
    Wt = np.matlib.rand(sh[0],10)
    Ht = np.matlib.rand(sh[1],10).T
    W.append(Wt);
    H.append(Ht);
    

In [89]:
print(W[0].shape)
print(H[0].shape)

(512, 10)
(10, 1474)


In [152]:
# update H
# 1474 x something

# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)
# H = list [10 x 1470][10 x 1470]
# view =  0

# up => (10 x 512 )(512 x 1474) => (10 x 1474)
# mid => (10  x 14704 )
# other => (10 x 512)(512x 10)(10 x 1470) => (10 x 1474)
# mid + other > (10 x 1470)
# final > (10 x 1470)


def h_coefficient_matrix(W,X,HF,alpha,beta,H,view_no):
    # Upper calculation
    up= 2*(W.T)*X # 512 x 1474 * 1474 x 512 => 512 x 512
    # for middle loop
    mid_H=0;
    for i in range(len(H)): # [H0, H1]
        if(i == view_no):
            continue;
        mid_H = mid_H + H[i];
    mid_H = alpha*mid_H;
    down = (2*W.T*(W)*HF)+2*beta*HF+mid_H; # H => previous H
    final = up/down;
    return final;

In [151]:
# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)
# H = list [10 x 1470][10 x 1470]
# view =  0

# up => 512x1474 ) (1474 x 10 ) => 512 x 10
# dow => 512 x 10 )( 10x1474 ) (1474 x 10) => 512 x 10

def w_cofficient_matrix(X,H,W):
    up = X * (H.T);
    down = W * H * (H.T)
    final = up/down
    return final;
    
    

In [145]:
from sklearn.metrics.pairwise import euclidean_distances
x = np.array([[1,2],[3,4.5]])
y = np.array([[1,2.1],[3,4]])
d = euclidean_distances(x,y)
di = [];
for i in range(2):
    di.append(d[i][i])
di
max(di)

0.5

In [124]:
a = V[0].shape
a[0]

512

In [153]:
# W = (512 x 10)
# X = (512 x 1474)
# HF = (10 x 1474)

def findError(X,W,H):
    loop = X.shape
    loop = loop[1];
    sumz = 0;
    L =W*H ; # 512 x 10 10x 1474 => 512 x 1474
    L = L.T;
    X = X.T;
    error = euclidean_distances(X,L);
    DI= [];
    for i in range(loop):
        DI.append(error[i][i])
    return max(DI)

In [159]:
for index,X in enumerate(V): # X1, X2
    while( findError(X,W[index],H[index]) >= 0.5): # 0.5 
        H[index] = h_coefficient_matrix(W[index],X,H[index],0.1,0.1,H,index);
        W[index] = w_cofficient_matrix(X,H[index],W[index]);
        #h_coefficient_matrix(W,X,HF,alpha,beta,H,view_no):
        print("Error::",findError(X,W[index],H[index]))
    print('End of View ',index)

Error:: 1.1677754780002148
Error:: 9.132640287603262
Error:: 16.13387331383481
Error:: 1.281953121190694
Error:: 8.524689012576234
Error:: 16.676471518000497
Error:: 1.1523122480402024
Error:: 9.045383877205943
Error:: 15.733426055326742
Error:: 1.2615300239547376
Error:: 8.687101142603082
Error:: 16.26441343516227
Error:: 1.1971120468822016
Error:: 9.207531988102996
Error:: 15.764827959081126
Error:: 1.2031738113920836
Error:: 9.04756989508856
Error:: 16.1826453527479
Error:: 1.2316750840773616
Error:: 9.446962428859516
Error:: 15.492073989180405
Error:: 1.2055819229104328
Error:: 9.024932041074448
Error:: 15.809803214962553
Error:: 1.175673164396408
Error:: 9.441375715676408
Error:: 15.187510769462532
Error:: 1.2766066087647352
Error:: 9.316407580342108
Error:: 15.35088474296939
Error:: 1.1772989016662423
Error:: 9.626410884136984
Error:: 15.133917398571485
Error:: 1.310904759707564
Error:: 9.503801662939184
Error:: 15.044740130340218
Error:: 1.200647107461596
Error:: 9.6135234422380

KeyboardInterrupt: 